In [4]:
from langgraph.pregel.remote import RemoteGraph
from langchain_core.messages import convert_to_messages
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
from langgraph_sdk import get_client

# Connect via SDK
url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

In [2]:
# Create a thread
thread = await client.threads.create()
thread

{'thread_id': '25f8ee9e-9cdd-4894-82e2-a3be35c36063',
 'created_at': '2025-07-09T01:40:22.206808+00:00',
 'updated_at': '2025-07-09T01:40:22.206808+00:00',
 'metadata': {},
 'status': 'idle',
 'config': {},
 'values': None,
 'interrupts': {},
 'error': None}

In [3]:
# Check any existing runs on a thread
thread = await client.threads.create()
runs = await client.runs.list(thread["thread_id"])
print(runs)

[]


In [6]:
# Ensure we've created some runs and saved them to my user_id
user_input = "What is 10 times that"
config = {"configurable": {"user_id": "Test"}}
graph_name = "agent" 
run = await client.runs.create(thread["thread_id"], graph_name, input={"messages": [HumanMessage(content=user_input)]}, config=config)

In [7]:
# Kick off a new thread and a new run
thread = await client.threads.create()
user_input = "Give me a summary of what I asked."
config = {"configurable": {"user_id": "Test"}}
graph_name = "agent" 
run = await client.runs.create(thread["thread_id"], graph_name, input={"messages": [HumanMessage(content=user_input)]}, config=config)

In [8]:
# Check the run status
print(await client.runs.get(thread["thread_id"], run["run_id"]))

{'run_id': '1f05c663-3e4e-688d-852a-9b0ee6939a25', 'thread_id': '14f75953-8e26-42dd-8054-43c0ece56d1f', 'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca', 'created_at': '2025-07-09T01:44:07.554691+00:00', 'updated_at': '2025-07-09T01:44:09.430001+00:00', 'metadata': {'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca'}, 'status': 'success', 'kwargs': {'input': {'messages': [{'id': None, 'name': None, 'type': 'human', 'content': 'Give me a summary of what I asked.', 'example': False, 'additional_kwargs': {}, 'response_metadata': {}}]}, 'config': {'metadata': {'created_by': 'system', 'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca'}, 'configurable': {'run_id': '1f05c663-3e4e-688d-852a-9b0ee6939a25', 'user_id': 'Test', 'graph_id': 'agent', 'thread_id': '14f75953-8e26-42dd-8054-43c0ece56d1f', 'user-agent': 'langgraph-sdk-py/0.1.72', 'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca', 'x-request-id': '4c67dbee-25ba-435c-9d19-efbb146a561f', '__after_seconds__': 0, 'langgrap

In [ ]:
user_input = "What are the arithemetic operations I asked you about"
async for chunk in client.runs.stream(thread["thread_id"], 
                                      graph_name, 
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      config=config,
                                      stream_mode="messages-tuple"):

    if chunk.event == "messages":
        print("".join(data_item['content'] for data_item in chunk.data if 'content' in data_item), end="", flush=True)